In [13]:
import pandas as pd
import yfinance as yf
import numpy as np
from finvizfinance.screener.overview import Overview
from scipy.stats import linregress
import time

In [14]:
# ==========================================
# 1.(Finviz Screening)
# ==========================================
def get_filtered_picks(limit_per_country=5000): # Raised limit to capture more
    print("--- STEP 1: Fetching Universe from Finviz ---")
    
    try:
        foverview = Overview()
        
        # Criteria: USA, Strong Buy, High Liquidity, Not Penny Stocks
        filters_dict = {
            
            'Country': 'USA',
            'Average Volume': 'Over 2M', 
            'Market Cap.': '+Small (over $300mln)',
            # 1. Institutional Floor (Removes Penny Stocks)
            'Price': 'Over $5',
    
            # 2. Legitimacy Check (Ensures Options Market exists)
            'Option/Short': 'Optionable',
    
            # 3. Solvency Check (Basic Liquidity) consider removing since it filters out telecom and utilities companies
            'Current Ratio': 'Over 1',
    
            # 4. Sector Exclusion (If you want to remove volatility) thinking about using later
            # 'Industry': 'Stocks only (ex-Funds)',

            #5 profitability filter
            'Operating Margin': 'Positive (>0%)',
            
            #6 Analyst Rating Filter consider changing later
            'Analyst Recom.': 'Strong Buy (1)',
        }
        foverview.set_filter(filters_dict=filters_dict)
        df_results = foverview.screener_view()
        
        if df_results.empty:
            return pd.DataFrame()
            
        print(f"   Success! Found {len(df_results)} initial candidates.")
        
        # Clean up Column Names immediately
        if 'Analyst Recom' in df_results.columns:
            df_results.rename(columns={'Analyst Recom': 'Recom'}, inplace=True)
            
        # Ensure numeric Price for later
        df_results['Price'] = pd.to_numeric(df_results['Price'], errors='coerce')
        
        return df_results

    except Exception as e:
        print(f"   Error in Finviz Step: {e}")
        return pd.DataFrame()

In [15]:
# ==========================================
# 1B. SPECIALIZED UNIVERSE: FINANCIALS (USA + CANADA)
# ==========================================
def get_financial_universe():
    print("--- STEP 1 (Financials): Fetching Banking & Insurance Universe (USA + Canada) ---")
    
    # Same logic: High volume for USA, Lower volume for dual-listed Canadian banks
    targets = [
        {'country': 'USA', 'vol': 'Over 2M'},      
        {'country': 'Canada', 'vol': 'Over 200K'}  
    ]
    
    combined_frames = []
    
    for target in targets:
        c_name = target['country']
        print(f"   Querying {c_name} Financial Sector...")
        
        try:
            foverview = Overview()
            
            # --- DISTINCT FILTERS FOR BANKS ---
            filters_dict = {
                'Country': c_name,
                'Sector': 'Financial',             # <--- CRITICAL: ONLY FINANCIALS
                'Average Volume': target['vol'], 
                'Market Cap.': '+Small (over $300mln)',
                'Price': 'Over $5',
                'Option/Short': 'Optionable',      # Quality filter
            }
            
            foverview.set_filter(filters_dict=filters_dict)
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                print(f"      -> Found {len(df_results)} financial stocks in {c_name}.")
                combined_frames.append(df_results)
            else:
                print(f"      -> No financial results for {c_name}.")
                
        except Exception as e:
            print(f"      Error fetching {c_name}: {e}")

    # Combine data
    if not combined_frames:
        return pd.DataFrame()
        
    fins_universe = pd.concat(combined_frames, ignore_index=True)
    
    # Clean up Column Names
    if 'Analyst Recom' in fins_universe.columns:
        fins_universe.rename(columns={'Analyst Recom': 'Recom'}, inplace=True)
        
    # Ensure numeric Price
    fins_universe['Price'] = pd.to_numeric(fins_universe['Price'], errors='coerce')
    
    # Remove duplicates
    fins_universe.drop_duplicates(subset=['Ticker'], inplace=True)
    
    print(f"   TOTAL FINANCIAL UNIVERSE: {len(fins_universe)} stocks.")
    return fins_universe

In [16]:
# ==========================================
# 2. THE CREDIT MODEL (Z-Score & Margins)
# ==========================================
def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score (Bankruptcy Risk)"""
    try:
        # Extract Key Metrics
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        
        # Z-Score Components
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        # Handle EBIT naming differences
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        # Ratios
        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        # Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E Altman Z-Score coefficients
        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan
#The Margin Trend column is a custom metric we built to measure operational momentum. 
# It tells you whether a company is becoming more or less efficient at making money over time.
def get_margin_trend(financials):
    """Calculates 3-Year Gross Margin Slope"""
    try:
        years = financials.columns[:3]
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        # Slope Calculation
        margins = margins[::-1] # Chronological order
        slope, _, _, _, _ = linregress(range(len(margins)), margins)
        
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"


In [17]:
# ==========================================
# 3. EXECUTION ENGINE (SPLIT BY TIER)
# ==========================================
import time

def run_screener_split():
    # A. Get the List from Step 1
    candidates = get_filtered_picks()
    
    if candidates.empty:
        print("No stocks found to analyze.")
        return None, None, None, None, None

    print(f"\n--- STEP 2: Running Credit Risk Model on {len(candidates)} Stocks ---")
    print("Separating stocks into distinct tiers...")
    
    # Initialize separate lists for each tier
    fortress_data = []
    moonshot_data = []
    distress_data = []
    financial_data = [] # Banks/Insurance (skipped by Z-score)
    middle_data = []    # Everything else
    
    # Loop through ALL candidates
    for index, row in candidates.iterrows():
        ticker = row['Ticker']
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. SEPARATE FINANCIALS
            # Z-Score doesn't work for banks, so we stash them here to analyze later
            sector = info.get('sector', 'Unknown')
            if 'Financial' in sector:
                financial_data.append({
                    'Ticker': ticker,
                    'Sector': sector,
                    'Price': row['Price'],
                    'Recom': row.get('Recom', 'N/A')
                })
                continue
            
            # 2. FETCH DATA
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                continue
            
            # 3. RUN METRICS (Using your helper functions)
            z_score = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            
            # Data packet to save
            stock_data = {
                'Ticker': ticker,
                'Company': row.get('Company', 'N/A'),
                'Z-Score': z_score,
                'Margin_Trend': trend, 
                'Price': row['Price'],
                'Recom': row.get('Recom', 'N/A'),
                'Sector': sector
            }
            
            # 4. SORT INTO LISTS (The Logic)
            if (z_score > 2.99) and (trend in ["Improving", "Stable"]):
                fortress_data.append(stock_data)
                
            elif (z_score < 1.8) and (trend == "Deteriorating"):
                distress_data.append(stock_data)
                
            elif (z_score < 1.8) and (trend == "Improving"):
                moonshot_data.append(stock_data)
            
            else:
                middle_data.append(stock_data)
            
            # Progress indicator
            if index % 10 == 0:
                print(f"   Processed {index} / {len(candidates)}...", end='\r')

        except Exception:
            continue

    # B. Convert Lists to DataFrames
    fortress_df = pd.DataFrame(fortress_data)
    moonshot_df = pd.DataFrame(moonshot_data)
    distress_df = pd.DataFrame(distress_data)
    financial_df = pd.DataFrame(financial_data)
    middle_df = pd.DataFrame(middle_data)
    
    print(f"\n\n--- ANALYSIS COMPLETE ---")
    print(f"Fortress (Safe):     {len(fortress_df)} stocks")
    print(f"Moonshot (Risky):    {len(moonshot_df)} stocks")
    print(f"Distress (Avoid):    {len(distress_df)} stocks")
    print(f"Financials (Skip):   {len(financial_df)} stocks")
    print(f"Middle of Road:      {len(middle_df)} stocks")
    
    return fortress_df, moonshot_df, distress_df, financial_df, middle_df

# --- RUN IT & UNPACK VARIABLES ---
# This saves the results into 5 separate dataframes immediately
fortress_df, moonshot_df, distress_df, fins_df, mid_df = run_screener_split()
#fins_df are banking and other financial sector stocks cannot use Z score due to customer deposits being liabilities

--- STEP 1: Fetching Universe from Finviz ---
   Success! Found 73 initial candidates.###--------] 3/4 

--- STEP 2: Running Credit Risk Model on 73 Stocks ---
Separating stocks into distinct tiers...
   Processed 70 / 73...

--- ANALYSIS COMPLETE ---
Fortress (Safe):     30 stocks
Moonshot (Risky):    6 stocks
Distress (Avoid):    3 stocks
Financials (Skip):   10 stocks
Middle of Road:      24 stocks


In [18]:
if fortress_df is not None and not fortress_df.empty:
    print("--- FORTRESS STOCKS (High Quality) ---")
    
    display(fortress_df.sort_values(by='Price', ascending=True))


--- FORTRESS STOCKS (High Quality) ---


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
25,TTI,"Tetra Technologies, Inc",3.22,Improving,9.02,N/A,Industrials
19,RELY,Remitly Global Inc,6.34,Improving,14.62,N/A,Technology
0,ADMA,Adma Biologics Inc,21.68,Improving,19.58,N/A,Healthcare
8,CWAN,Clearwater Analytics Holdings Inc,32.28,Improving,22.25,N/A,Technology
6,CPNG,Coupang Inc,4.04,Improving,23.20,N/A,Consumer Cyclical
15,MIR,Mirion Technologies Inc,3.43,Improving,23.55,N/A,Industrials
21,SLB,SLB Ltd,3.06,Improving,38.08,N/A,Energy
3,APG,APi Group Corporation,3.07,Improving,38.88,N/A,Industrials
20,SEI,Solaris Energy Infrastructure Inc,4.88,Improving,44.96,N/A,Energy
4,BROS,Dutch Bros Inc,4.42,Improving,64.83,N/A,Consumer Cyclical


In [19]:
#high risk picks
if moonshot_df is not None and not moonshot_df.empty:
    print("--- MOONSHOT STOCKS (Speculative) ---")
    display(moonshot_df.sort_values(by='Price', ascending=True))

--- MOONSHOT STOCKS (Speculative) ---


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
2,MNKD,Mannkind Corp,-7.24,Improving,5.84,N/A,Healthcare
5,XERS,Xeris Biopharma Holdings Inc,-0.31,Improving,7.34,N/A,Healthcare
4,PRMB,Primo Brands Corp,0.91,Improving,16.10,N/A,Consumer Defensive
0,ET,Energy Transfer LP,1.34,Improving,16.39,N/A,Energy
1,MGNI,Magnite Inc,0.73,Improving,16.40,N/A,Communication Services
3,PGY,Pagaya Technologies Ltd,1.30,Improving,22.85,N/A,Technology


In [20]:
#very high risk / avoid picks
if distress_df is not None and not distress_df.empty:
    print("--- DISTRESS (Avoid / Short Candidates) ---")
    display(distress_df.sort_values(by='Z-Score', ascending=True))

--- DISTRESS (Avoid / Short Candidates) ---


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
0,BULL,Webull Corp,-0.47,Deteriorating,8.57,N/A,Technology
2,PAYO,Payoneer Global Inc,0.23,Deteriorating,5.80,N/A,Technology
1,CWH,Camping World Holdings Inc,1.64,Deteriorating,10.00,N/A,Consumer Cyclical


In [21]:
#mid picks for analysis
mid_df
if mid_df is not None and not mid_df.empty:
    print("--- Mid Picks requires more analysis ---")
    display(mid_df.sort_values(by='Z-Score', ascending=True))

--- Mid Picks requires more analysis ---


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
2,BCRX,Biocryst Pharmaceuticals Inc,-2.54,Stable,7.41,N/A,Healthcare
8,DD,DuPont de Nemours Inc,0.48,Stable,41.26,N/A,Basic Materials
15,GEO,"Geo Group, Inc",1.61,Stable,15.98,N/A,Industrials
7,DAR,Darling Ingredients Inc,1.85,Deteriorating,34.37,N/A,Consumer Defensive
5,CENX,Century Aluminum Co,1.95,Improving,36.21,N/A,Basic Materials
13,ESI,Element Solutions Inc,2.14,Improving,25.16,N/A,Basic Materials
20,RPRX,Royalty Pharma plc,2.33,N/A,38.60,N/A,Healthcare
4,BTU,Peabody Energy Corp,2.43,Deteriorating,30.87,N/A,Energy
17,LFST,LifeStance Health Group Inc,2.50,Improving,7.05,N/A,Healthcare
22,WAY,Waystar Holding Corp,2.80,Deteriorating,32.84,N/A,Healthcare


In [22]:
# ==========================================
# 4. SPECIALIZED FINANCIALS ENGINE (FIXED)
# ==========================================
def run_financials_analysis(df):
    """
    Takes the raw list of Financial stocks (Banks/Insurance),
    fetches valuation metrics, and filters for Value & Income.
    """
    if df is None or df.empty:
        print("No financial stocks to analyze.")
        return

    print(f"--- ANALYZING {len(df)} FINANCIAL STOCKS (USA + TSX) ---")
    print("Fetching P/E, P/B, Dividend, and Analyst Ratings...\n")
    
    bank_data = []
    
    for index, row in df.iterrows():
        ticker = row['Ticker']
        try:
            # Fetch Data
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. VALUATION METRICS
            pe = info.get('trailingPE', np.nan)
            pb = info.get('priceToBook', np.nan)
            
            # 2. EFFICIENCY & INCOME
            roe = info.get('returnOnEquity', np.nan)
            div_yield = info.get('dividendYield', 0)
            if div_yield is None: div_yield = 0
            
            # 3. ANALYST RATING (The Fix)
            # We fetch directly from YF to ensure we get a number (1=Strong Buy, 5=Sell)
            recom = info.get('recommendationMean', None)
            
            # Fallback: If no analyst covers it, assume 3.0 (Hold) so it doesn't crash
            if recom is None: 
                recom = 3.0

            # 4. GROWTH
            rev_growth = info.get('revenueGrowth', np.nan)

            # Save it
            bank_data.append({
                'Ticker': ticker,
                'Company': row.get('Company', ticker),
                'Price': row['Price'],
                'P/E': pe,
                'P/B': pb,
                'ROE': roe,
                'Yield%': round(div_yield * 100, 2),
                'Recom': float(recom),  # Ensure it is a float for filtering
                'Rev_Growth': rev_growth,
                'Sector': row.get('Sector', 'Financial')
            })
            
        except Exception:
            continue
            
    # Create the Enriched DataFrame
    rich_fins_df = pd.DataFrame(bank_data)
    
    if rich_fins_df.empty:
        print("Could not fetch details for financial stocks.")
        return

    # ==========================================
    # FILTER 1: "UNDERVALUED BANKING" (The Value Play)
    # Criteria: Cheap (P/E < 15), Trading near assets (P/B < 1.2), Profitable (ROE > 8%)
    # ==========================================
    value_picks = rich_fins_df[
        (rich_fins_df['P/E'] < 15) & 
        (rich_fins_df['P/B'] < 1.2) & 
        (rich_fins_df['ROE'] > 0.08)
    ].copy()
    
    print(f"--- FILTER 1: UNDERVALUED BANKS (P/E < 15 & P/B < 1.2) ---")
    if not value_picks.empty:
        display(value_picks.sort_values(by='P/B', ascending=True))
    else:
        print("No stocks met the strict value criteria.")

    # ==========================================
    # FILTER 2: "INCOME COMPOUNDERS" (The Dividend Play)
    # Criteria: High Yield (>2.5%), Growing (>0%), Sustainable P/E (<20)
    # PLUS: Analyst Buy Rating (Score <= 2.5)
    # ==========================================
    income_picks = rich_fins_df[
        (rich_fins_df['Yield%'] >= 2.5) & 
        (rich_fins_df['P/E'] < 20) &
        (rich_fins_df['ROE'] > 0.05) &
        (rich_fins_df['Recom'] <= 2.0)  #consider removing or changing later
    ].copy()

    print(f"\n--- FILTER 2: INCOME GENERATORS (Yield > 2.5% & Buy Rating) ---")
    if not income_picks.empty:
        display(income_picks.sort_values(by='Yield%', ascending=False))
    else:
        print("No stocks met the dividend criteria.")

    return rich_fins_df

In [23]:
# ==========================================
# MAIN EXECUTION: FINANCIALS TRACK
# ==========================================

# 1. Fetch the data (Uses the function from Part 1B)
# Note: This checks USA (>2M Volume) and Canada (>200K Volume)
fins_df = get_financial_universe()

# 2. Analyze the data (Uses the function from Part 4)
# This will display the tables for "Undervalued Banks" and "Income Generators"
if not fins_df.empty:
    final_banks_df = run_financials_analysis(fins_df)
else:
    print("No financial stocks found to analyze.")

--- STEP 1 (Financials): Fetching Banking & Insurance Universe (USA + Canada) ---
   Querying USA Financial Sector...
      -> Found 85 financial stocks in USA.###------] 4/5 
   Querying Canada Financial Sector...
      -> Found 9 financial stocks in Canada.
   TOTAL FINANCIAL UNIVERSE: 94 stocks.
--- ANALYZING 94 FINANCIAL STOCKS (USA + TSX) ---
Fetching P/E, P/B, Dividend, and Analyst Ratings...

--- FILTER 1: UNDERVALUED BANKS (P/E < 15 & P/B < 1.2) ---


,Ticker,Company,Price,P/E,P/B,ROE,Yield%,Recom,Rev_Growth,Sector
56,MARA,MARA Holdings Inc,10.18,3.961090,0.747156,0.23066,0.0,1.92308,0.917,Financial
13,BMNR,BitMine Immersion Technologies Inc,31.36,2.342046,0.846744,0.08016,3.0,1.00000,0.942,Financial
61,OBDC,Blue Owl Capital Corp,12.65,8.971631,0.849392,0.09772,893.0,3.00000,0.116,Financial
4,ARCC,Ares Capital Corp,19.94,10.020101,0.996850,0.10061,963.0,1.53333,0.009,Financial
17,BXSL,Blackstone Secured Lending Fund,27.37,10.328301,1.008252,0.10021,1125.0,3.00000,0.045,Financial
62,ONB,Old National Bancorp,23.35,13.735294,1.129438,0.08265,240.0,2.00000,0.478,Financial



--- FILTER 2: INCOME GENERATORS (Yield > 2.5% & Buy Rating) ---


,Ticker,Company,Price,P/E,P/B,ROE,Yield%,Recom,Rev_Growth,Sector
4,ARCC,Ares Capital Corp,19.94,10.020101,0.996850,0.10061,963.0,1.53333,0.009,Financial
77,USB,U.S. Bancorp,54.16,12.393593,1.491066,0.11720,377.0,2.00000,0.072,Financial
79,VLY,Valley National Bancorp,12.02,13.659092,0.918188,0.07067,366.0,1.64286,0.263,Financial
44,HBAN,"Huntington Bancshares, Inc",17.69,12.370630,1.322815,0.10434,350.0,1.66667,0.148,Financial
36,FITB,Fifth Third Bancorp,47.83,14.277613,1.634934,0.11511,335.0,1.81818,0.067,Financial
65,PNC,PNC Financial Services Group Inc,210.20,13.587588,1.549325,0.11486,324.0,1.91304,0.108,Financial
21,CFG,Citizens Financial Group Inc,59.19,16.579832,1.076730,0.06714,311.0,1.52381,0.136,Financial
57,MET,Metlife Inc,81.57,15.361582,1.862074,0.12798,278.0,1.76471,-0.059,Financial
91,RY,Royal Bank Of Canada,168.74,16.526936,2.554993,0.15295,278.0,1.75000,0.138,Financial
38,FNB,F.N.B. Corp,17.76,12.685715,0.959119,0.07870,270.0,1.37500,0.105,Financial


In [24]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND', 'ARCC'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 2 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...


C:\Users\jdcc3\AppData\Local\Temp\ipykernel_12748\3242308346.py:46: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)



--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
1,GRND,13.89,1.46,17.74,-44.73,1.40,21.75,1.07,3.19
0,ARCC,19.94,-0.99,20.48,-10.94,1.27,22.64,2.04,1.38
